---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [3]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-10-17 19:30:47--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.10.78, 188.185.33.206, 188.185.22.33, ...
Connecting to zenodo.org (zenodo.org)|188.185.10.78|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2023-10-17 19:30:48--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin? 100%[===================>]   1.35G  34.6MB/s    in 42s     

2023-10-17 19:31:31 (32.9 MB/s) - ‘SO_vectors_200.bin?download=1.1’ saved [1453905423/1453905423]



In [4]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [5]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [6]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [7]:
def most_similar(target_word, embeddings, topn=5):

    # Получаем топ ближайших слов
    top_similar_words = embeddings.most_similar(target_word, topn=topn)

    # Проверяем, входит ли 'cat' в топ ближайших слов и его позицию
    is_cat_in_top = False
    cat_position = None
    for idx, (word, _) in enumerate(top_similar_words):
        if word == 'cat':
            is_cat_in_top = True
            cat_position = idx + 1
            break

    return top_similar_words, is_cat_in_top, cat_position


# Пример использования
word = 'dog'
top_words, is_cat_in_top, cat_pos = most_similar(word, wv_embeddings, 5)

print(f"Топ-5 близких слов к слову '{word}':")
for idx, (w, score) in enumerate(top_words, 1):
    print(f"{idx}. {w}: {score:.4f}")

Топ-5 близких слов к слову 'dog':
1. animal: 0.8564
2. dogs: 0.7881
3. mammal: 0.7624
4. cats: 0.7621
5. animals: 0.7608


**Видим, что в нашем топе присутствует "cats" вместо "cat".**

**В техническом смысле, "cat" и "cats" рассматриваются как разные токены, и, следовательно, они имеют разные векторные представления в большинстве моделей word embeddings.**

**Однако семантически "cats" очень близко к "cat", и в некоторых контекстах они могут считаться эквивалентными. Поэтому, думаю можно считать, что "cat" занимает 4 место.**

In [8]:
# проверим это для twitter, так как в нём используются более сленговые, краткие и эмоциональные выражения

import gensim.downloader as api
twitter = api.load('glove-twitter-100')

In [9]:
word = 'dog'
if word in twitter:
    print(twitter[word].dtype, twitter[word].shape)

float32 (100,)


In [10]:
print(f"Num of words: {len(twitter.index_to_key)}")

Num of words: 1193514


In [11]:
word = 'dog'
top_words, is_cat_in_top, cat_pos = most_similar(word, twitter, 5)

print(f"Топ-5 близких слов к слову '{word}':")
for idx, (w, score) in enumerate(top_words, 1):
    print(f"{idx}. {w}: {score:.4f}")

Топ-5 близких слов к слову 'dog':
1. cat: 0.8752
2. dogs: 0.8713
3. puppy: 0.8299
4. pet: 0.8060
5. cats: 0.7908


**Семантическая близость:** Слово "cat" является наиболее близким словом к "dog" среди всех слов в топ-5. Это указывает на высокую семантическую близость между словами "dog" и "cat". Это ожидаемо, так как оба этих слова обозначают домашних питомцев и часто упоминаются в схожих контекстах.

**Множественное число:** Слова "dogs" и "cats" также входят в топ-5, что показывает, что модель также улавливает варианты слов в множественном числе.

**Синонимы и связанные слова:** Слова "puppy" (щенок) и "pet" (питомец) также находятся в топе, что говорит о том, что модель правильно определяет синонимы и слова, тесно связанные по смыслу.

**Качество векторных представлений:** Тот факт, что "cat" и "dogs" занимают верхние позиции в ранжированном списке, демонстрирует качество векторных представлений, обученных на основе семантической близости слов.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

#MyTokenizer

 Метод tokenize класса MyTokenizer разбивает текст на слова, состоящие из букв, цифр и символов подчеркивания, и возвращает их в виде списка. Такой принцип токенизации не учитывает апострофы и другие символы, что может быть не идеально в некоторых контекстах. Например, слово "How's" будет разбито на два токена: "How" и "s".

In [67]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()
tokens = tokenizer.tokenize("This isn't my book. It's my sister's.")
print(tokens)

['This', 'isn', 't', 'my', 'book', 'It', 's', 'my', 'sister', 's']


**Такой результат может быть неидеальным для некоторых задач, потому что он разбивает слова и сокращения на отдельные части, что может потерять часть семантического значения.**

**Word2Vec**

In [44]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    # Токенизируем вопрос
    tokens = tokenizer.tokenize(question)

    # Инициализируем счетчик слов и вектор для суммирования
    word_count = 0
    question_vec = np.zeros(dim)

    for token in tokens:
        if token in embeddings:
            question_vec += embeddings[token]
            word_count += 1

    # Если было хотя бы одно слово в embeddings, делаем средний вектор
    if word_count > 0:
        question_vec /= word_count

    return question_vec

Теперь у нас есть метод для создания векторного представления любого предложения.

In [45]:
# Пример

question_to_vec("I love neural networks", wv_embeddings, MyTokenizer())

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114,
        0.28913016,  1.03125   , -2.86546615, -1.31928726,  2.20982948,
        1.16786722,  1.08419518,  1.84672268, -0.98733809,  0.87209455,
       -0.17219625,  0.73839017, -0.78278264, -0.88278695, -0.61526108,
       -0.15872356, -0.61707981,  0.36668165,  2.01375335,  0.26274722,
       -1.4950124 , -1.38671508, -1.20499369, -1.01661696,  2.76211715,
        1.72557124, -0.19038178, -0.58994474, -0.1141155 , -1.88903529,
       -1.78835048, -0.20420383, -0.70620341,  2.4890339 , -0.15452717,
        1.12550129,  0.26108352,  1.31532462, -0.57667526, -0.33271411,
        1.67649019,  0.91229995, -0.6988014 ,  2.37525521, -1.76342229,
        3.13780514,  1.22432033, -1.05544141,  0.78975705,  0.04404024,
        1.16230452, -0.44780231, -0.9149816 ,  0.01501505, -0.85523087,
       -1.01834532, -0.71621163,  1.0727205 ,  0.88818762, -0.29

**Twitter**

In [46]:
def question_to_vec_twitter(question, embeddings, tokenizer, dim=100):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    # Токенизируем вопрос
    tokens = tokenizer.tokenize(question)

    # Инициализируем счетчик слов и вектор для суммирования
    word_count = 0
    question_vec = np.zeros(dim)

    for token in tokens:
        if token in embeddings:
            question_vec += embeddings[token]
            word_count += 1

    # Если было хотя бы одно слово в embeddings, делаем средний вектор
    if word_count > 0:
        question_vec /= word_count

    return question_vec

In [47]:
question_to_vec_twitter("I love neural networks", twitter, MyTokenizer())

array([ 0.16183334, -0.052978  ,  0.04376013, -0.10697334,  0.01288134,
       -0.09652967, -0.16466999, -0.43725034,  0.29546466, -0.20336667,
        0.21300801, -0.32862134, -2.75922334, -0.57549999, -0.53619333,
       -0.29053999, -0.30203967, -0.11751666, -0.24970333,  0.24262399,
       -0.41314001,  0.35261501,  0.11756644,  0.48368667, -0.25788333,
        0.18926334, -0.33232   ,  0.42995167, -0.20675133,  0.06934666,
       -0.33527999, -0.55339999, -0.04456743, -0.71956331, -0.05976672,
       -0.19774867,  0.40228335,  0.44025001, -0.00489667, -0.130009  ,
        0.09617333,  0.13951466,  0.15943667,  0.10783667,  0.121478  ,
        0.62786334, -0.15943733, -0.13058668,  0.16010332, -0.309998  ,
        0.22485667, -0.21194766, -0.11315527, -0.03986001,  0.05037666,
        0.29896334, -0.93921332, -0.50247999, -0.33547334,  0.10249233,
        0.032346  , -0.05446534, -0.66603334,  0.17085399,  0.13502533,
        0.19593133, -0.16092668,  0.53488334, -0.40200901, -0.11

#NLTKTokenizer

In [57]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [68]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string

class NLTKTokenizer:
    def __init__(self):
        self.punctuations = set(string.punctuation)
        self.tokenizer = RegexpTokenizer(r"\w+(?:'\w+)?")

    def tokenize(self, text):
        return self.tokenizer.tokenize(text)

tokenizer_nltk = NLTKTokenizer()
tokens = tokenizer_nltk.tokenize("This isn't my book. It's my sister's.")
print(tokens)

['This', "isn't", 'my', 'book', "It's", 'my', "sister's"]


#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

**Word2Vec**

In [71]:
# Преобразуем предложение в векторное представление
sentence_vec = question_to_vec("I love neural networks", wv_embeddings, MyTokenizer())

# Получим третью компоненту и округлим
third_component = round(sentence_vec[2], 2)
third_component

-1.29

**Twitter**

In [48]:
# Преобразуем предложение в векторное представление
sentence_vec = question_to_vec_twitter("I love neural networks", twitter, MyTokenizer())

# Получим третью компоненту и округлим
third_component = round(sentence_vec[2], 2)
third_component

0.04

**Наиболее полезным способом использования таких векторов является сравнение их друг с другом для определения семантической близости предложений или слов. Одно измерение в отдельности не дает много информации, но в сочетании со всеми другими измерениями оно помогает определить уникальное векторное представление для каждого слова или предложения.**

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

Hits@K:
Эта метрика вычисляет, находится ли дублирующий вопрос среди первых K позиций ранжированного списка для каждого исходного вопроса.

DCG@K:
Эта метрика учитывает порядок ответов в ранжированном списке, штрафуя ответы с более высоким рангом.

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Максимальное значение Hits@47Hits@47 равно 1 (если дубликат действительно находится в топ-47).**

**Максимальное значение DCG@1DCG@1 равно 1 (если дубликат действительно на первой позиции).**

**Таким образом, максимальное значение разницы Hits@47 - DCG@1Hits@47−DCG@1 равно 0.**

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

[K = 9]  DCG@1 = 1/(log2(1+9)) * [9≤10] = 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [16]:
from math import log2

In [17]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    return sum(1 for rank in dup_ranks if rank <= k) / len(dup_ranks)

In [18]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    return sum(1 / (log2(1 + rank)) for rank in dup_ranks if rank <= k) / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [19]:
import pandas as pd

In [20]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(copy_answers[0]) + 1]

# вычисляем метрику для разных k
data = {
    'HITS': [hits_count(dup_ranks, k) for k in range(1, 5)],
    'DCG': [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)]
}

df = pd.DataFrame(data).T
df.columns = [1, 2, 3, 4]
df

,1,2,3,4
HITS,0.0,1.00000,1.00000,1.00000
DCG,0.0,0.63093,0.63093,0.63093


У вас должно получиться

In [21]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [22]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [23]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [24]:
validation_data = read_corpus(r'/content/data/validation.tsv')
for item in validation_data[:5]:
    print(item)

How to print a binary heap tree without recursion?	How do you best convert a recursive function to an iterative one?	How can i use ng-model with directive in angular js	flash: drawing and erasing	toggle react component using hide show classname	Use a usercontrol from another project to current webpage	~ Paths resolved differently after upgrading to ASP.NET 4	Materialize datepicker - Rendering when an icon is clicked	Creating PyPi package - Could not find a version that satisfies the requirement iso8601	How can I analyze a confusion matrix?	How do I declare a C array in Swift?	Using rand() when flipping a coin and rolling a die	Handling a JSON field with a special character in its name in Java	React Native select row on ListView when push it	Get 'creation_time' of video using ffmpeg and regex	Does row exist and multiple where	How to specify a classifier in a gradle dependency's dependency?	Using $unwind on multiple documents	Visual Studio Code: Missing X509Certificate2UI	Developing Cord

Кол-во строк

In [25]:
len(validation_data)

3760

Размер нескольких первых строк

In [26]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

**Word2Vec**

In [28]:
# def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
#     question_vec = question_to_vec(question, embeddings, tokenizer, dim)
#     candidate_vecs = np.array([question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates])

#     # Вычисление косинусного сходства между векторами вопроса и кандидатов
#     cosine_similarities = cosine_similarity([question_vec], candidate_vecs)[0]

#     # Сопоставление начальных позиций кандидатов с их косинусными сходствами
#     initial_positions = list(range(len(candidates)))

#     # Сортировка кандидатов по их косинусному сходству
#     sorted_candidates = sorted(zip(initial_positions, candidates), key=lambda x: cosine_similarities[x[0]], reverse=True)

#     return sorted_candidates


def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    # Преобразование входного вопроса в векторное представление с использованием embeddings
    question_vec = question_to_vec(question, embeddings, tokenizer)

    # Преобразование каждого кандидата в векторное представление
    # Результатом будет список векторных представлений кандидатов
    cand_vecs = [question_to_vec(candidate, embeddings, tokenizer) for candidate in candidates]

    similarities = []

    # Для каждого векторного представления кандидата вычисляем косинусное сходство с векторным представлением вопроса
    for i, cand_vec in enumerate(cand_vecs):
        # Вычисление косинусного сходства между вектором вопроса и вектором кандидата
        similarity = cosine_similarity([question_vec], [cand_vec])[0][0]
        similarities.append((candidates[i], similarity))

    result_df = pd.DataFrame(similarities, columns=['candidate', 'similarity'])
    result_df = result_df.sort_values(by='similarity', ascending=False)

    return result_df

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [29]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [30]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

                                           candidate  similarity
1           C# create cookie from string and send it    0.546291
0  Convert Google results object (pure js) to Pyt...    0.478863
2      How to use jQuery AJAX for an outside domain?    0.161885

                                           candidate  similarity
1  WPF- How to update the changes in list item of...    0.476891
0  Getting all list items of an unordered list in...    0.456955
2              select2 not displaying search results    0.163290



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

**Twitter**

In [49]:
def rank_candidates_twitter(question, candidates, embeddings, tokenizer, dim=200):
    # Преобразование входного вопроса в векторное представление с использованием embeddings
    question_vec = question_to_vec_twitter(question, embeddings, tokenizer)

    # Преобразование каждого кандидата в векторное представление
    # Результатом будет список векторных представлений кандидатов
    cand_vecs = [question_to_vec_twitter(candidate, embeddings, tokenizer) for candidate in candidates]

    similarities = []

    # Для каждого векторного представления кандидата вычисляем косинусное сходство с векторным представлением вопроса
    for i, cand_vec in enumerate(cand_vecs):
        # Вычисление косинусного сходства между вектором вопроса и вектором кандидата
        similarity = cosine_similarity([question_vec], [cand_vec])[0][0]
        similarities.append((candidates[i], similarity))

    result_df = pd.DataFrame(similarities, columns=['candidate', 'similarity'])
    result_df = result_df.sort_values(by='similarity', ascending=False)

    return result_df

In [51]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates_twitter(question, q_candidates, twitter, tokenizer)
        print(ranks)
        print()

                                           candidate  similarity
2      How to use jQuery AJAX for an outside domain?    0.859483
1           C# create cookie from string and send it    0.847946
0  Convert Google results object (pure js) to Pyt...    0.791999

                                           candidate  similarity
1  WPF- How to update the changes in list item of...    0.677805
0  Getting all list items of an unordered list in...    0.646960
2              select2 not displaying search results    0.631718



In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

**Видим, что с использованием Word2Vec embeddings результаты получились лучше, чем с twitter. Поэтому дальше будет использовать именно Word2Vec**

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ: 102**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [32]:
from tqdm.notebook import tqdm

**Word2Vec**

---



In [33]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('	')
    q = line[0]  # извлечение вопросов
    ex = line[1:]  # извлечение кандидатов
    ranks = rank_candidates(q, ex , wv_embeddings, tokenizer)
    wv_ranking.append(ranks[ranks['candidate']==ex[0]].index[0]+1)  # поиск рейтинга правильного ответа

  0%|          | 0/3760 [00:00<?, ?it/s]

In [34]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.989 | Hits@   1: 0.989
DCG@   5: 0.989 | Hits@   5: 0.989
DCG@  10: 0.989 | Hits@  10: 0.989
DCG@ 100: 0.989 | Hits@ 100: 0.989
DCG@ 500: 0.990 | Hits@ 500: 0.993
DCG@1000: 0.990 | Hits@1000: 1.000


**Twitter**

In [52]:
twitter_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('	')
    q = line[0]  # извлечение вопросов
    ex = line[1:]  # извлечение кандидатов
    ranks = rank_candidates_twitter(q, ex , twitter, tokenizer)
    twitter_ranking.append(ranks[ranks['candidate']==ex[0]].index[0]+1)  # поиск рейтинга правильного ответа

  0%|          | 0/3760 [00:00<?, ?it/s]

In [53]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(twitter_ranking, k), k, hits_count(twitter_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.988 | Hits@   1: 0.988
DCG@   5: 0.988 | Hits@   5: 0.988
DCG@  10: 0.988 | Hits@  10: 0.988
DCG@ 100: 0.988 | Hits@ 100: 0.989
DCG@ 500: 0.989 | Hits@ 500: 0.993
DCG@1000: 0.989 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [35]:
train_data = read_corpus(r'/content/data/train.tsv')

In [36]:
for item in train_data[:5]:
    print(item)

converting string to list	Convert Google results object (pure js) to Python object

Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?	Event handling for geometries in Three.js?

Sending array via Ajax fails	Getting all list items of an unordered list in PHP

How to insert CookieCollection to CookieContainer?	C# create cookie from string and send it

Updating one element of a bound Observable collection	WPF- How to update the changes in list item of a list



Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

**Параметр window в модели Word2Vec определяет максимальное расстояние между текущим и прогнозируемым словом в предложении. Чем больше окно, тем больше контекст слова учитывается. Думаю, что лучший размер окна находится в диапазоне от 5 до 10.**

In [37]:
words = []
for line in train_data:
    for sentence in line.split('\t'):
        words.append(sentence.split())
words

[['converting', 'string', 'to', 'list'],
 ['Convert',
  'Google',
  'results',
  'object',
  '(pure',
  'js)',
  'to',
  'Python',
  'object'],
 ['Which',
  'HTML',
  '5',
  'Canvas',
  'Javascript',
  'to',
  'use',
  'for',
  'making',
  'an',
  'interactive',
  'drawing',
  'tool?'],
 ['Event', 'handling', 'for', 'geometries', 'in', 'Three.js?'],
 ['Sending', 'array', 'via', 'Ajax', 'fails'],
 ['Getting',
  'all',
  'list',
  'items',
  'of',
  'an',
  'unordered',
  'list',
  'in',
  'PHP'],
 ['How', 'to', 'insert', 'CookieCollection', 'to', 'CookieContainer?'],
 ['C#', 'create', 'cookie', 'from', 'string', 'and', 'send', 'it'],
 ['Updating',
  'one',
  'element',
  'of',
  'a',
  'bound',
  'Observable',
  'collection'],
 ['WPF-',
  'How',
  'to',
  'update',
  'the',
  'changes',
  'in',
  'list',
  'item',
  'of',
  'a',
  'list'],
 ['MongoDB', 'error', 'on', 'find()'],
 ['Retrieve',
  'only',
  'the',
  'queried',
  'element',
  'in',
  'an',
  'object',
  'array',
  'in',
  'M

In [54]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,      # data for model to train on
                 vector_size=200,         # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [55]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('	')
    q = line[0]  # извлечение вопросов
    ex = line[1:]  # извлечение кандидатов
    ranks = rank_candidates(q, ex , embeddings_trained, tokenizer)
    wv_ranking.append(ranks[ranks['candidate']==ex[0]].index[0]+1)  # поиск рейтинга правильного ответа

  0%|          | 0/3760 [00:00<?, ?it/s]

In [40]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.989 | Hits@   1: 0.989
DCG@   5: 0.989 | Hits@   5: 0.989
DCG@  10: 0.989 | Hits@  10: 0.989
DCG@ 100: 0.989 | Hits@ 100: 0.989
DCG@ 500: 0.990 | Hits@ 500: 0.993
DCG@1000: 0.990 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


#**Тестирование на вводе пользователем**

In [83]:
# Подготовим множество всех уникальных предложений из train_data
all_sentences = set()
for line in train_data:
    for sentence in line.split('\t'):
        all_sentences.add(sentence)
candidates = [list(all_sentences)]

In [84]:
user_input = input("Введите ваше предложение: ")

questions = [user_input]

for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, embeddings_trained, tokenizer)

ranks.head()

Введите ваше предложение: How to create pie using matplotlib?


,candidate,similarity
1329661,How to create stacked barchart using matplotlib,0.982531
1192672,How to create a draggable legend in matplotlib?\n,0.916527
1314346,How to create pie chart using javascript,0.912962
243438,How do I create a pie chart using Bokeh?,0.909029
383816,How to create bubble tooltips in PyQt?\n,0.908642


In [82]:
user_input = input("Введите ваше предложение: ")

questions = [user_input]

for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, embeddings_trained, tokenizer)

ranks.head()

Введите ваше предложение: How ctreate dataframe in pandas?


,candidate,similarity
982900,How to split dataframe in pandas,0.948267
1582872,How to pivot a dataframe in Pandas?,0.939061
969168,How to downsample a pandas dataframe\n,0.935826
125892,How to plot a heatmap from pandas DataFrame\n,0.914879
1345356,How to compute jaccard similarity from a panda...,0.913836
